In [127]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import pandas as pd


In [128]:
# read csv with pandas
subjective = pd.read_csv('evaluation/subjective/analysis_test.csv')
speech = pd.read_csv('evaluation/objective/metrics_avg_std.csv')
pstsm = pd.read_csv('evaluation/objective/tsmps/metrics_avg_std.csv')

In [129]:
subjective_ps = subjective[subjective['trial_type']== 'ps']
subjective_ps.loc[subjective_ps['stimulus'] == 'anchor', 'stimulus'] = 'psola'
subjective_ps = subjective_ps.sort_values(by=['stimulus'])
subjective_ps

,trial_type,stimulus,mean
1,ps,librosaps,2.7520
2,ps,nmps,2.0608
0,ps,psola,2.3912
3,ps,pvps,3.6172


In [130]:
subjective_ts = subjective[subjective['trial_type']== 'tsm']
# rename 'stimulus' == anchor to 'res'
subjective_ts.loc[subjective_ts['stimulus'] == 'anchor', 'stimulus'] = 'res'
subjective_ts = subjective_ts.sort_values(by=['stimulus'])
subjective_ts

,trial_type,stimulus,mean
5,tsm,hps,2.4444
6,tsm,nm,2.5160
7,tsm,pv,3.6720
4,tsm,res,2.7444
8,tsm,tsmnet,2.0392


In [131]:
speech = speech.drop(columns=[col for col in speech.columns if "STD" in col])
speech_ps = speech[speech["algo"].isin(["PSOLA", "NMPS", "LibrosaPS", "PVPS"])]
speech_ps = speech_ps.sort_values(by=['algo'])
speech_ps


,algo,STOI,PESQ,VISQOL,SISNR,NISQA TTS,NISQA MOS,NISQA NOISE,NISQA DIST,NISQA COL,NISQA LOUD,AA CE,AA AU,AA PC,AA PQ
2,LibrosaPS,0.760365,1.777249,2.553176,-21.215640,2.759496,2.748352,3.509001,3.536018,3.101451,3.465523,3.791741,4.416189,1.826860,4.972952
1,NMPS,0.713321,1.502706,2.440833,-15.230104,2.740925,2.675556,3.473221,3.564485,3.148291,3.539138,3.553002,3.528271,2.287931,4.116749
0,PSOLA,0.693266,1.394415,2.462580,-20.138571,2.479378,2.894725,3.650726,3.352680,3.448860,3.710702,4.961367,5.779078,1.677189,6.101228
3,PVPS,0.716958,2.584315,2.776234,-23.897644,3.056075,3.279870,3.685009,3.847269,3.555808,3.866438,4.204177,4.885843,1.827949,5.377454


In [132]:
# The method name should be 'isin', not 'isnatin'
speech_ts = speech[speech["algo"].isin(["HPS", "PV", "TSMNET", "NM", "RES"])]
speech_ts = speech_ts.sort_values(by=['algo'])

speech_ts

,algo,STOI,PESQ,VISQOL,SISNR,NISQA TTS,NISQA MOS,NISQA NOISE,NISQA DIST,NISQA COL,NISQA LOUD,AA CE,AA AU,AA PC,AA PQ
6,HPS,0.812284,1.852370,2.548224,-17.982173,2.848507,3.273374,3.748411,3.648217,3.688808,3.882528,4.487943,5.369333,1.639039,6.175137
8,NM,0.644734,1.494516,2.441115,-18.093646,3.019876,2.926365,3.676661,3.197247,3.368198,3.667829,3.605795,3.820379,2.049372,4.905680
7,PV,0.893820,2.779820,2.810307,-18.622277,3.114009,3.581602,3.817769,3.737843,3.764793,4.026639,4.442584,5.372234,1.637074,6.121521
4,RES,0.999766,4.641333,4.992663,40.179171,3.180123,3.474355,3.744241,3.957474,3.756633,3.930550,4.296323,5.222179,1.740651,6.142637
5,TSMNET,0.526281,1.211070,1.923298,-0.610866,2.292489,1.823896,3.676049,2.710686,2.176944,2.719807,3.147230,3.540797,1.633503,4.167374


In [133]:
pstsm = pstsm.drop(columns=[col for col in pstsm.columns if "STD" in col])
# change all values in SE-MSE column to negative
pstsm['SE-MSE'] = -1 * pstsm['SE-MSE']
pstsm['DRV'] = -1 * np.abs(pstsm['DRV'])


ps_ps = pstsm[pstsm["algo"].isin(["PSOLA", "NMPS", "LibrosaPS", "PVPS"])]
ps_ps = ps_ps.sort_values(by=['algo'])
ps_ps

,algo,DRV,SE-MSE
3,LibrosaPS,-2.205693,-213.698845
2,NMPS,-2.125251,-251.286868
0,PSOLA,-2.132164,-207.865765
1,PVPS,-3.616632,-95.201805


In [134]:
ps_ts = pstsm[pstsm["algo"].isin(["HPS", "PV", "TSMNET", "NM", "RES"])]
ps_ts = ps_ts.sort_values(by=['algo'])
ps_ts

,algo,DRV,SE-MSE
4,HPS,-1.878646,-33.868699
7,NM,-2.445098,-96.134082
5,PV,-2.022402,-20.006227
8,RES,-0.029743,-0.189229
6,TSMNET,-0.547299,-707.617629


In [135]:
# Merge subjective and objective metrics for PS
merged_ps = subjective_ps.copy()
merged_ps['stimulus'] = merged_ps['stimulus'].str.lower()
ps_map = {'psola': 'PSOLA', 'nmps': 'NMPS', 'librosaps': 'LibrosaPS', 'pvps': 'PVPS'}
merged_ps['algo'] = merged_ps['stimulus'].map(ps_map)
merged_ps = merged_ps.merge(ps_ps, on='algo')


# Merge subjective and objective metrics for TSM
merged_ts = subjective_ts.copy()
merged_ts['stimulus'] = merged_ts['stimulus'].str.lower()
ts_map = {'hps': 'HPS', 'nm': 'NM', 'pv': 'PV', 'res': 'RES', 'tsmnet': 'TSMNET'}
merged_ts['algo'] = merged_ts['stimulus'].map(ts_map)
merged_ts = merged_ts.merge(ps_ts, on='algo')


In [ ]:
# Save DRV and SE-MSE correlation results to CSV
drv_semse_results = []

# PS correlations
for metric in ['DRV', 'SE-MSE']:
    pearson_corr = stats.pearsonr(merged_ps['mean'], merged_ps[metric])
    spearman_corr = stats.spearmanr(merged_ps['mean'], merged_ps[metric])
    drv_semse_results.append({
        'Type': 'PS',
        'Metric': metric,
        'Pearson_r': pearson_corr[0],
        'Pearson_p': pearson_corr[1],
        'Spearman_r': spearman_corr.correlation,
        'Spearman_p': spearman_corr.pvalue
    })

# TSM correlations
for metric in ['DRV', 'SE-MSE']:
    pearson_corr = stats.pearsonr(merged_ts['mean'], merged_ts[metric])
    spearman_corr = stats.spearmanr(merged_ts['mean'], merged_ts[metric])
    drv_semse_results.append({
        'Type': 'TSM',
        'Metric': metric,
        'Pearson_r': pearson_corr[0],
        'Pearson_p': pearson_corr[1],
        'Spearman_r': spearman_corr.correlation,
        'Spearman_p': spearman_corr.pvalue
    })

drv_semse_df = pd.DataFrame(drv_semse_results)
drv_semse_df.to_csv('evaluation/subjective/drv_semse_correlation_results.csv', index=False)

In [ ]:
# Prepare to collect results
results = []

# PS correlations
for metric in ['STOI', 'PESQ', 'VISQOL', 'SISNR', 'NISQA TTS', 'NISQA MOS', 'NISQA NOISE', 'NISQA DIST', 'NISQA COL', 'NISQA LOUD', 'AA CE', 'AA AU', 'AA PC', 'AA PQ']:
    merged = subjective_ps.copy()
    merged['algo'] = merged['stimulus'].map(ps_map)
    merged = merged.merge(speech_ps, on='algo')
    pearson_corr = stats.pearsonr(merged['mean'], merged[metric])
    spearman_corr = stats.spearmanr(merged['mean'], merged[metric])
    results.append({
        'Type': 'PS',
        'Metric': metric,
        'Pearson_r': pearson_corr[0],
        'Pearson_p': pearson_corr[1],
        'Spearman_r': spearman_corr.correlation,
        'Spearman_p': spearman_corr.pvalue
    })
    print(f"PS {metric}: Pearson r={pearson_corr[0]:.3f}, p={pearson_corr[1]:.3f}; Spearman r={spearman_corr.correlation:.3f}, p={spearman_corr.pvalue:.3f}")

print("===========================")

# TSM correlations
for metric in ['STOI', 'PESQ', 'VISQOL', 'SISNR', 'NISQA TTS', 'NISQA MOS', 'NISQA NOISE', 'NISQA DIST', 'NISQA COL', 'NISQA LOUD', 'AA CE', 'AA AU', 'AA PC', 'AA PQ']:
    merged = subjective_ts.copy()
    merged['algo'] = merged['stimulus'].map(ts_map)
    merged = merged.merge(speech_ts, on='algo')
    pearson_corr = stats.pearsonr(merged['mean'], merged[metric])
    spearman_corr = stats.spearmanr(merged['mean'], merged[metric])
    results.append({
        'Type': 'TSM',
        'Metric': metric,
        'Pearson_r': pearson_corr[0],
        'Pearson_p': pearson_corr[1],
        'Spearman_r': spearman_corr.correlation,
        'Spearman_p': spearman_corr.pvalue
    })
    print(f"TSM {metric}: Pearson r={pearson_corr[0]:.3f}, p={pearson_corr[1]:.3f}; Spearman r={spearman_corr.correlation:.3f}, p={spearman_corr.pvalue:.3f}")


# Save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('evaluation/subjective/correlation_results.csv', index=False)

PS STOI: Pearson r=0.208, p=0.792; Spearman r=0.600, p=0.400
PS PESQ: Pearson r=0.955, p=0.045; Spearman r=0.800, p=0.200
PS VISQOL: Pearson r=0.987, p=0.013; Spearman r=1.000, p=0.000
PS SISNR: Pearson r=-0.912, p=0.088; Spearman r=-1.000, p=0.000
PS NISQA TTS: Pearson r=0.781, p=0.219; Spearman r=0.800, p=0.200
PS NISQA MOS: Pearson r=0.897, p=0.103; Spearman r=0.800, p=0.200
PS NISQA NOISE: Pearson r=0.665, p=0.335; Spearman r=0.800, p=0.200
PS NISQA DIST: Pearson r=0.787, p=0.213; Spearman r=0.400, p=0.600
PS NISQA COL: Pearson r=0.613, p=0.387; Spearman r=0.400, p=0.600
PS NISQA LOUD: Pearson r=0.668, p=0.332; Spearman r=0.400, p=0.600
PS AA CE: Pearson r=0.131, p=0.869; Spearman r=0.400, p=0.600
PS AA AU: Pearson r=0.303, p=0.697; Spearman r=0.400, p=0.600
PS AA PC: Pearson r=-0.468, p=0.532; Spearman r=-0.200, p=0.800
PS AA PQ: Pearson r=0.340, p=0.660; Spearman r=0.400, p=0.600
TSM STOI: Pearson r=0.657, p=0.228; Spearman r=0.800, p=0.104
TSM PESQ: Pearson r=0.462, p=0.433; Spe